In [1]:
# Step 1: Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import seaborn as sns
from sklearn.metrics import confusion_matrix
import random


In [2]:
# Step 2: Set paths and hyperparameters
data_dir = r"E:\\pathmnist"
image_size = 64
batch_size = 64
num_epochs = 20

In [3]:
# Step 3: Define image transformations
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

In [4]:
# Step 5: Define Deeper CNN Model
class DeepCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeepCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout(0.25),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout(0.25),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout(0.4),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 8 * 8, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [5]:
# Step 6: Initialize Model
train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transform)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=val_test_transform)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=val_test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

num_classes = len(train_data.classes)
class_names = train_data.classes
model = DeepCNN(num_classes).to(device)

Using device: cpu


In [6]:
# Label smoothing loss
def smooth_cross_entropy(preds, target, smoothing=0.1):
    confidence = 1.0 - smoothing
    log_probs = torch.nn.functional.log_softmax(preds, dim=-1)
    true_dist = torch.zeros_like(log_probs)
    true_dist.fill_(smoothing / (preds.size(1) - 1))
    true_dist.scatter_(1, target.data.unsqueeze(1), confidence)
    return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [ ]:
# Step 7: Train the Model
train_accs, val_accs, train_losses, val_losses = [], [], [], []
best_val_acc = 0
patience = 6
trigger_times = 0

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = smooth_cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    scheduler.step()

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)
    train_accs.append(train_acc)
    train_losses.append(train_loss)

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += smooth_cross_entropy(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

Epoch 1/20 | Train Acc: 76.21% | Val Acc: 89.05%
Epoch 2/20 | Train Acc: 92.09% | Val Acc: 94.11%
Epoch 3/20 | Train Acc: 94.05% | Val Acc: 95.64%
Epoch 4/20 | Train Acc: 94.96% | Val Acc: 94.38%
Epoch 5/20 | Train Acc: 95.50% | Val Acc: 97.96%
Epoch 6/20 | Train Acc: 96.93% | Val Acc: 97.50%
